# Exercises

This notebook contains exercises for section 2.1 of the Medford group graduate student training. These exercises cover basic DFT calculations.

## Exercise 1

We begin with single point calculations in SPARC. Compute gas-phase formation energies for water, carbon dioxide, and ammonia using the common Perdew–Burke-Ernzerhof generalized gradient approximation (GGA) functional. You can get the reference energy for carbon from graphite. Compare your results to those from experimentation. What do you notice?

In [67]:
import os
from ase import Atoms, io
from ase.io import read, write
from ase.build import bulk, molecule, surface, add_adsorbate
from ase.units import Bohr,Hartree,mol,kcal,kJ,eV
from sparc import SPARC

cwd = os.getcwd()
E_dict = {}

mol_list = ['H2O', 'CO2', 'NH3', 'H2', 'O2', 'N2', 'graphite']
for mol in mol_list:
    
    if mol not in ['graphite']:
        atoms = molecule(mol)
        atoms.cell = [[10,0,0],[0,10,0],[0,0,10]]
        atoms.pbc = True

    parameters = dict(
                    EXCHANGE_CORRELATION = 'GGA_PBE',
                    D3_FLAG=1,   #Grimme D3 dispersion correction
                    SPIN_TYP=1,   #spin-polarized calculation
                    KPOINT_GRID=[1,1,1],  
                    ECUT=800/Hartree,   #set ECUT (Hartree) or h (Angstrom)
                    #h = 0.15,
                    TOL_SCF=1e-4,
                    RELAX_FLAG=1,
                    TOL_RELAX = 5.00E-04,  #convergence criteria (maximum force) (Ha/Bohr)
                    PRINT_FORCES=1,
                    PRINT_RELAXOUT=1)
    
    dir_i = f"{cwd}/Exercise_1/PBE/{mol}"
    parameters['directory'] = dir_i
    
    if os.path.isdir(dir_i):
        print(f"{dir_i} already exists.")
        continue
    
    if mol == 'H2':
        parameters['SPIN_TYP']=0
    elif mol == 'graphite':
        atoms = read('graphite.cif')
        parameters['KPOINT_GRID'] = [6,6,2]
    
    calc = SPARC(atoms = atoms, **parameters)
    atoms.set_calculator(calc)

    eng = atoms.get_potential_energy()
    E_dict[mol] = eng
    
    with open(f'{dir_i}/energy.txt', 'w') as f:
        f.write(str(eng))
    



/home/hice1/nyu49/Data/VIP/Excercises/Exercises_DFT_basics/Exercise_1/PBE/H2O already exists.
/home/hice1/nyu49/Data/VIP/Excercises/Exercises_DFT_basics/Exercise_1/PBE/CO2 already exists.
/home/hice1/nyu49/Data/VIP/Excercises/Exercises_DFT_basics/Exercise_1/PBE/NH3 already exists.
/home/hice1/nyu49/Data/VIP/Excercises/Exercises_DFT_basics/Exercise_1/PBE/H2 already exists.
/home/hice1/nyu49/Data/VIP/Excercises/Exercises_DFT_basics/Exercise_1/PBE/O2 already exists.
/home/hice1/nyu49/Data/VIP/Excercises/Exercises_DFT_basics/Exercise_1/PBE/N2 already exists.


/home/hice1/nyu49/.local/lib/python3.9/site-packages/sparc/calculator.py:471: UserWarning: You have specified one of FD_GRID, ECUT or MESH_SPACING, conversion of h to mesh grid is ignored.
  warn("You have specified one of FD_GRID, ECUT or MESH_SPACING, "
srun: Job 11957 step creation temporarily disabled, retrying (Requested nodes are busy)
srun: Step created for job 11957


Step  0
Step  1
Step  2
{}
[0, 1, 2, 3] [0, 1, 2, 3]


/home/hice1/nyu49/.local/lib/python3.9/site-packages/sparc/sparc_parsers/utils.py:56: UserWarning: Key POISSON_SOLVER not in validator's parameter list, ignore value conversion!
  warn(
/home/hice1/nyu49/.local/lib/python3.9/site-packages/sparc/sparc_parsers/utils.py:56: UserWarning: Key VERBOSITY not in validator's parameter list, ignore value conversion!
  warn(


In [68]:
import re, json

def readFile(path):
    f = open(path, 'r')
    content = f.readlines()
    return content

def read_energy(path):
    energy = float(readFile(path)[0])
    return energy

def get_E_form(name):
    E_elem = {}
    E_elem['H'] = E_dict['H2']/2
    E_elem['O'] = E_dict['O2']/2
    E_elem['C'] = E_dict['graphite'] / len(read('graphite.cif'))
    E_elem['N'] = E_dict['N2']/2
    counts = mol_dict[name].symbols.formula.count()    
    E_form = E_dict[name]
    
    #print(counts, E_form)
    for elem in counts:
        #print(counts[elem], E_elem[elem])
        E_form += -counts[elem] * E_elem[elem]
    return E_form 

E_dict, mol_dict, t_dict = {}, {}, {}
for mol in mol_list:
    
     ### get energy dictionary
    dir_mol = f'Exercise_1/PBE/{mol}'
    E_dict[mol] = read_energy(f'{dir_mol}/energy.txt')
    
     ### get Atoms dictionary
    try:
        mol_dict[mol] = molecule(mol)    
    except KeyError:
        pass
    mol_dict['graphite'] = read('graphite.cif')
    
     ### get time dictionary
    sparc_out = os.path.join(dir_mol, 'SPARC.out')
    with open(sparc_out, 'r') as f: data = f.read()
    t_SPARC = float(re.search(r'Total walltime\s+:\s+([0-9.]+)', data).group(1)) 
    t_SPARC = float(f"{t_SPARC:.3f}")
    t_dict[mol] = t_SPARC
    

In [70]:
E_form_dict_Expt = json.load(open('E_form_dict_Expt.json', 'r'))

for mol in mol_list:
    E_form = get_E_form(mol)
    if E_form != 0:
        print(f"\u0394E_form ({mol}). Calculation: {E_form:.3f} eV ({t_dict[mol]} sec to calculate {mol})")
        print(f"\t        Experiment: {E_form_dict_Expt[mol]:.3f}")




ΔE_form (H2O). Calculation: -2.518 eV (8.542 sec to calculate H2O)
	        Experiment: -2.713
ΔE_form (CO2). Calculation: -3.903 eV (16.004 sec to calculate CO2)
	        Experiment: -4.128
ΔE_form (NH3). Calculation: -1.061 eV (10.887 sec to calculate NH3)
	        Experiment: -0.816


Repeat the above calculations using the B3LYP hydrid functional. How do these energies compare to those from experimentation and from DFT with the PBE functional? Does the computation time with the B3LYP change? Explain why or why not.
 # # Not completed due to sparc-dft-api error

In [37]:
import os
from ase import Atoms, io
from ase.io import read, write
from ase.build import bulk, molecule, surface, add_adsorbate
from ase.units import Bohr,Hartree,mol,kcal,kJ,eV
from sparc import SPARC

cwd = os.getcwd()
E_dict = {}

mol_list = ['H2O', 'CO2', 'NH3', 'H2', 'O2', 'N2', 'graphite']
for mol in mol_list:
    
    if mol not in ['graphite']:
        atoms = molecule(mol)
        atoms.cell = [[10,0,0],[0,10,0],[0,0,10]]
        atoms.pbc = True

    parameters = dict(
                    EXCHANGE_CORRELATION = 'PBE0',
                    D3_FLAG=1,   #Grimme D3 dispersion correction
                    SPIN_TYP=1,   #spin-polarized calculation
                    KPOINT_GRID=[1,1,1],  
                    ECUT=800/Hartree,   #set ECUT (Hartree) or h (Angstrom)
                    #h = 0.15,
                    TOL_SCF=1e-4,
                    RELAX_FLAG=1,
                    TOL_RELAX = 5.00E-04,  #convergence criteria (maximum force) (Ha/Bohr)
                    PRINT_FORCES=1,
                    PRINT_RELAXOUT=1)
    
    dir_i = f"{cwd}/Exercise_1/PBE0/{mol}"
    parameters['directory'] = dir_i
    
    if os.path.isdir(dir_i):
        print(f"{dir_i} already exists.")
        continue
    
    if mol == 'H2':
        parameters['SPIN_TYP']=0
    elif mol == 'graphite':
        atoms = read('graphite.cif')
    
    calc = SPARC(atoms = atoms, **parameters)
    atoms.set_calculator(calc)

    eng = atoms.get_potential_energy()
    E_dict[mol] = eng
    
    with open(f'{dir_i}/energy.txt', 'w') as f:
        f.write(str(eng))
    


Step  0
Step  1
Step  2
Step  3


IndexError: list index out of range

## Exercise 2

Next, we revisit Ca-LTA from the prior ASE exercises. Use DFT with the PBE functional to perform a single point energy calculation for the empty Ca-LTA framework. Report the resulting energy in eV/atom.

In [7]:
import os
from ase import Atoms, io
from ase.io import read, write
from ase.build import bulk, molecule, surface, add_adsorbate
from ase.units import Bohr,Hartree,mol,kcal,kJ,eV
from sparc import SPARC

cwd = os.getcwd()

for setting in ['NoD3_NoSpin', 'D3_NoSpin', 'D3_Spin']:

    parameters = dict(
                    EXCHANGE_CORRELATION = 'GGA_PBE',
                    D3_FLAG=0,   #Grimme D3 dispersion correction
                    SPIN_TYP=0,   #spin-polarized calculation
                    KPOINT_GRID=[1,1,1],  
                    ECUT=600/Hartree,   #set ECUT (Hartree) or h (Angstrom)
                    #h = 0.15,
                    TOL_SCF=1e-4,
                    RELAX_FLAG=0,
                    TOL_RELAX = 5.00E-04,  #convergence criteria (maximum force) (Ha/Bohr)
                    PRINT_FORCES=1,
                    PRINT_RELAXOUT=1)
    
    dir_i = f"{cwd}/Exercise_2/{setting}"
    parameters['directory'] = dir_i
    
    if setting == 'NoD3_NoSpin':
        parameters['D3_FLAG'] = 0
        parameters['SPIN_TYP'] = 0 
    elif setting == 'D3_NoSpin':
        parameters['D3_FLAG'] = 1 
        parameters['SPIN_TYP'] = 0 
    elif setting == 'D3_Spin':
        parameters['D3_FLAG'] = 1 
        parameters['SPIN_TYP'] = 1 
        
    if os.path.isdir(dir_i):
        print(f"{dir_i} already exists.")
        continue
    
    atoms = read('CaLTA.vasp')
    atoms.pbc = True
    
    calc = SPARC(atoms = atoms, **parameters)
    atoms.set_calculator(calc)

    eng = atoms.get_potential_energy()
    print(f"{setting}: {eng}")

{}
[73, 55, 56, 57, 58, 59, 60, 61, 54, 62, 72, 64, 65, 66, 67, 68, 63, 69, 70, 71, 3, 0, 1, 2, 42, 47, 43, 44, 45, 46, 53, 48, 49, 50, 51, 52, 39, 40, 41, 38, 18, 16, 15, 14, 13, 12, 11, 17, 10, 9, 8, 7, 6, 19, 28, 20, 36, 35, 34, 33, 32, 31, 30, 37, 29, 27, 26, 25, 24, 23, 22, 21, 5, 4] [21, 22, 23, 20, 73, 72, 52, 51, 50, 49, 48, 46, 45, 44, 43, 42, 41, 47, 40, 53, 55, 71, 70, 69, 68, 67, 66, 65, 54, 64, 62, 61, 60, 59, 58, 57, 56, 63, 39, 36, 37, 38, 24, 26, 27, 28, 29, 25, 31, 32, 33, 34, 35, 30, 8, 1, 2, 3, 4, 5, 6, 7, 9, 16, 11, 12, 13, 14, 15, 17, 18, 19, 10, 0]
NoD3_NoSpin: -26353.786194078464
{}
[73, 55, 56, 57, 58, 59, 60, 61, 54, 62, 72, 64, 65, 66, 67, 68, 63, 69, 70, 71, 3, 0, 1, 2, 42, 47, 43, 44, 45, 46, 53, 48, 49, 50, 51, 52, 39, 40, 41, 38, 18, 16, 15, 14, 13, 12, 11, 17, 10, 9, 8, 7, 6, 19, 28, 20, 36, 35, 34, 33, 32, 31, 30, 37, 29, 27, 26, 25, 24, 23, 22, 21, 5, 4] [21, 22, 23, 20, 73, 72, 52, 51, 50, 49, 48, 46, 45, 44, 43, 42, 41, 47, 40, 53, 55, 71, 70, 69, 68,

Repeat the above calculation with both spin polarization and a D3 dispersion correction with Becke-Johnson damping. How does the answer change and why?
# # NO D3-BJ in SPARC

## Exercise 3

Follow the steps from the previous set of exercises to import Ca-LTA as an Atoms object with periodic boundary conditions. But this time, use LiH5C5N2O5 structure for faster calculation.

In [73]:
from ase import Atoms, io
from ase.io import read, write

atoms = read('LiH5C5N2O5.cif')
atoms.pbc = True

print(len(atoms))

18


Use an ASE calculator to compute the potential energy (Do single-point calculation, which means that you don't need to do atomic relaxation) for LiH5C5N2O5 in eV/atom.

In [74]:
from sparc import SPARC
from ase.units import Bohr,Hartree,mol,kcal,kJ,eV

parameters = dict(
                EXCHANGE_CORRELATION = 'GGA_PBE',
                D3_FLAG=1,   #Grimme D3 dispersion correction
                SPIN_TYP=0,   #non spin-polarized calculation
                KPOINT_GRID=[2,2,1],  #slab needs only 1 kpt in z-direction 
                ECUT=500/Hartree,   #set ECUT (Hartree) or h (Angstrom)
                #h = 0.15,
                TOL_SCF=1e-4,
                RELAX_FLAG=0,
                TOL_RELAX = 3.00E-03,  #convergence criteria (maximum force) (Ha/Bohr)
                PRINT_FORCES=1,
                PRINT_RELAXOUT=1)

parameters['directory'] = 'Exercise_3'

calc = SPARC(atoms = atoms, **parameters)
atoms.set_calculator(calc)

eng = atoms.get_potential_energy()
print(eng)

srun: Job 11957 step creation temporarily disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 11957 step creation still disabled, retrying (Requested nodes are busy)
srun: Job 

{}
[10, 5, 6, 7, 8, 9, 1, 2, 0, 4, 3, 11, 12, 13, 14, 15, 16, 17] [8, 6, 7, 10, 9, 1, 2, 3, 4, 5, 0, 11, 12, 13, 14, 15, 16, 17]
-3911.4133248926873


/home/hice1/nyu49/.local/lib/python3.9/site-packages/sparc/sparc_parsers/out.py:108: UserWarning: Key atomic mass from run information appears multiple times in your outputfile!
  warn(


Next, manipulate the ASE Atoms object by increasing each cell lattice parameter by 20%. Be sure to scale atomic positions when setting the cell parameters. Visualize the expanded unit cell using ASE tools.

In [ ]:
cell0 = atoms.get_cell()

atoms2 = atoms.copy()
atoms2.set_cell(cell0*1.2, scale_atoms=True)  
print('original:', cell0.lengths())
print('new:', atoms2.get_cell().lengths())

Perform full optimization (both atomic positions and cell) in ASE starting from the expanded unit cell. Use the BFGS algorithm and print the trajectory to "opt.traj". Confirm that the lattice parameters relax to approximately the values from the original structure.

### # Currently, ase optimizers do not work well with sparc-dft-api.
### # So, this solution doesn't create "opt.traj."
### # There is an error with RELAX_FLAG=2

In [ ]:
atoms3 = atoms2.copy()

parameters['RELAX_FLAG'] = 2
#calc = SPARC(atoms = atoms3, **parameters)
calc = EMT()

atoms3.set_calculator(calc)

eng = atoms3.get_potential_energy()
print(eng)
print(atoms3.get_cell().lengths())